In [6]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext

In [109]:
#variables
price_sold = []
city_sold = []
state_sold = []
zip_code = []
bds = []
ba = []
sqft = []
broker_name = []
month_sold = []
year_sold = []

In [110]:
def scrape_zillow(city, state):
    """Scrapes Zillow for sold houses for the given city and state.
    
    Args:
        city (str): city name
        state (str): state abbreviation
            
    Returns: N/A
    """
    #for avoiding Captchas
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    
    with requests.Session() as s:
        url = 'https://www.zillow.com/' + city + '-' + state + '/sold/'
        r = s.get(url, headers=req_headers)
    
    #creating BeautifulSoup object
    soup = BeautifulSoup(r.content, 'html.parser')

    #scraping for each variable type
    price = soup.find_all(class_='list-card-price')
    address = soup.find_all (class_= 'list-card-addr')
    beds = list(soup.find_all("ul", class_="list-card-details"))
    broker = soup.find_all ('div', {'class': 'list-card-footer'})
    link = soup.find_all (class_= 'list-card-link')
    sold = soup.find_all(class_ = 'list-card-variable-text list-card-img-overlay')
    
    #Adds data to each variable list
    for x in price:
        price_sold.append(x.get_text().strip())
    for y in address:
        city_sold.append(city)
        state_sold.append(state)
        add = y.get_text().strip()
        zip_code.append(add[len(add) - 5 : len(add)])
    for z in beds:
        details = z.get_text().strip()
        if 'bds' in details:
            bds.append(details.split("bds",1)[0])
        else:
            bds.append(0)
            
        if ('ba' in details) and ('bds' in details):
            ba.append(details.split("ba",1)[0].split("bds",1)[1])
        else:
            ba.append(0)
            
        if ('sqft' in details) and ('ba' in details):
            sqft.append(details.split("sqft",1)[0].split("ba",1)[1])
        elif 'sqft' in details:
            sqft.append(details.split("sqft",1)[0])
        else:
            sqft.append(0)                        
    for a in broker[0:len(broker) - 1]:
        broker_name.append(a.get_text().strip())
    for b in sold:
        date = b.get_text().strip()
        month_sold.append(date[5:7])
        year_sold.append(date[len(date) - 4: len(date)])


In [243]:
#List of biggest cities in each of the 5 states below
florida = ['Jacksonville', 'Miami', 'Tampa', 'Orlando', 'Hialeah', 'Tallahassee', 'Hollywood',
           'Gainesville', 'Miramar', 'Clearwater', 'Brandon', 'Lakeland', 'Riverview', 'Davie',
           'Sunrise', 'Deltona', 'Alafaya', 'Plantation', 'Melbourne', 'Largo', 'Homestead', 
           'Kendall', 'Kissimmee']
georgia = ['Atlanta', 'Columbus', 'Savannah', 'Roswell', 'Albany', 'Marietta', 'Stonecrest',
           'Smyrna', 'Valdosta', 'Brookhaven', 'Dunwoody', 'Newnan', 'Gainesville', 'Milton', 
           'Mableton', 'Rome', 'Tucker', 'Woodstock', 'Hinesville', 'Douglasville', 
           'Evans', 'Martinez', 'Dalton']
alabama = ['Huntsville', 'Birmingham', 'Montgomery', 'Mobile', 'Tuscaloosa', 'Hoover', 'Auburn', 
           'Dothan', 'Decatur', 'Madison', 'Florence', 'Prattville', 'Gadsden', 'Alabaster', 'Northport', 
           'Opelika', 'Enterprise', 'Daphne', 'Homewood', 'Trussville', 'Bessemer', 'Athens', 'Pelham']
south_carolina = ['Charleston', 'Columbia', 'Greenville', 'Sumter', 'Florence', 'Spartanburg', 
                  'Greer', 'Aiken', 'Anderson', 'Bluffton', 'Conway', 'Maulten', 'Lexington', 
                  'Simpsonville', 'Taylors', 'Easley', 'Greenwood', 'Socastee', 'Hanahan', 'Clemson',
                  'Berea', 'Ladson', 'Gantt', 'Cayce', 'Parker']
tennessee = ['Memphis', 'Knoxville', 'Chattanooga', 'Clarksville', 'Franklin', 'Jackson',
             'Hendersonville', 'Bartlett', 'Kingsport', 'Smyrna', 'Cleveland', 'Brentwood', 'Gallatin',
             'Germantown', 'Lebanon', 'Cookeville', 'Maryville', 'Morristown', 'Bristol', 'Shelbyville',
             'Farragut', 'Tullahoma', 'Springfield', 'Sevierville', 'Dickson']



In [113]:
import time
import random

#scrapes for data on sold houses in the cities of each of the 5 southern states
#pauses for random interval between 5-10 seconds to avoid being blocked
for city in florida:
    scrape_zillow(city, 'fl')
    time.sleep(random.randint(5, 10))

for city in georgia:
    scrape_zillow(city, 'ga')
    time.sleep(random.randint(5, 10))
    
for city in alabama:
    scrape_zillow(city, 'al')
    time.sleep(random.randint(5, 10))
    
for city in south_carolina:
    scrape_zillow(city, 'sc')
    time.sleep(random.randint(5, 10))
    
for city in tennessee:
    scrape_zillow(city, 'tn')
    time.sleep(random.randint(5, 10))

In [258]:
#putting together the DataFrame
df_homes = pd.DataFrame()

df_homes['price_sold'] = price_sold
df_homes['city_sold'] = city_sold
df_homes['state_sold'] = state_sold
df_homes['zip_code'] = zip_code
df_homes['bds'] = bds
df_homes['ba'] = ba
df_homes['sqft'] = sqft
df_homes['broker_name'] = broker_name
df_homes['month_sold'] = month_sold
df_homes['year_sold'] = year_sold

In [260]:
df_homes

,price_sold,city_sold,state_sold,zip_code,bds,ba,sqft,broker_name,month_sold,year_sold
0,"$232,600",Jacksonville,fl,32234,3,2,"1,406",,10,2107
1,"$495,000",Jacksonville,fl,32081,3,2,"1,832",,11,2021
2,"$473,500",Jacksonville,fl,32256,4,4,"2,360",,11,2021
3,"$343,000",Jacksonville,fl,32257,3,2,"1,728",,11,2021
4,"$300,000",Jacksonville,fl,32221,4,2,"1,952",,11,2021
...,...,...,...,...,...,...,...,...,...,...
1057,"$255,000",Dickson,tn,37055,3,2,"1,225",,10,2021
1058,"$220,000",Dickson,tn,37055,3,1,"1,107",,10,2021
1059,"$280,000",Dickson,tn,37029,3,2,"1,366",,10,2021
1060,"$290,000",Dickson,tn,37055,3,2,"1,283",,10,2021


In [265]:
df_homes.to_csv('res/real_estate_prices_in_the_south.csv', index=False)